In [ ]:
!pip install --quiet langchain-core langchain-openai tiktoken openai

In [ ]:
import re, os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.messages import HumanMessage
OPENAI_API_KEY=""
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# 1. 게임 설정

## 1.1 라이어 게임의 테마와 제시어 선택

In [ ]:
llm_selector = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=1,          # 쉽게 이해될 난이도 유지
    request_timeout=30,
    api_key=os.environ["OPENAI_API_KEY"],
)

fewshot_prompt = """
아래는 ‘테마’와 ‘제시어’(구체 단어) 한 쌍을 보여 주는 예시입니다.

테마: 과일
제시어: 사과

테마: 영화 장르
제시어: 코미디

테마: 직업
제시어: 교사

테마: 유명 건축물
제시어: 에펠탑

---
위와 비슷한 난이도로 새로운 한 쌍을 한국어로 생성하세요.
다른 설명 없이 정확히 다음 형식만 출력하십시오.

테마: <새 테마>
제시어: <해당 제시어>
""".strip()

raw = llm_selector.invoke([HumanMessage(content=fewshot_prompt)]).content.strip()

m = re.search(r"테마:\s*(.+)\s+제시어:\s*(.+)", raw)
if not m:
    raise ValueError("LLM 출력에서 테마/제시어를 찾을 수 없습니다.")

THEME, WORD = m.group(1).strip(), m.group(2).strip()

print(f"선택된 테마: {THEME}")
print(f"선택된 제시어: {WORD}")

선택된 테마: 스포츠
선택된 제시어: 축구


## 1.2 공개 대화 기록 리스트 CONV_LOG 생성

In [ ]:
CONV_LOG = []

## 1.3 플레이어1 생성

In [ ]:
# 페르소나 프롬프트
PLAYER1_SYSTEM = f"""
당신은 ‘liar 게임’ 일반 참가자다.
- 테마: {THEME}          # 예: 장소
- 제시어: {WORD}         # 라이어는 이 줄이 없다
참가자 중에는 테마는 알고있지만 제시어는 알지 못하는 플레이어가 있습니다.

규칙
1. 매 턴, 한 명에게 한 번씩 질문한다. 질문받은 사람은 반드시 답한다.
2. 질문·답변은 “제시어를 알고 있다”는 뉘앙스를 풍기되, 라이어가 단서를 잡지 못하게 암시적으로 표현한다. 제시어를 직언(직접 명시)하는 것은 금지.
3. 턴 종료 후 모두가 라이어로 의심하는 플레이어 한 명에게 투표한다.
   · 최다 득표자가 라이어면 즉시 게임 종료 ─ 일반 플레이어 승리.
   · 라이어가 아니면 탈락 후 게임 지속.
4. 라이어는 마지막 두 명이 될 때까지 정체가 들키지 않으면 단독 승리한다.

행동 지침
- 자신이 누구든(라이어/일반) 정체를 숨긴 채 자연스럽게 대화하라.
- 논리적이되 과감한 블러핑·회피·거짓말을 적절히 활용하되, 규칙은 반드시 준수한다.
- 모든 발화는 한두 문장 안에서 명확하고 일관성 있게 작성한다.
- 당신은 이 게임의 논리가형 플레이어로 라이어를 논리적으로 게임을 이끌어나갑니다.
"""
# 플레이어 LLM
llm_p1 = ChatOpenAI(model="gpt-4o-mini",
                    temperature=0.7,
                    request_timeout=60,
                    api_key=OPENAI_API_KEY)

prompt_p1 = ChatPromptTemplate.from_messages([
    ("system", PLAYER1_SYSTEM.strip()),
    ("human", """
[지금까지의 대화 기록]
{history}

[지시사항]
{user_input}
""")
])

# 질문 전략 프롬프트
question_strategy_prompt_p1 = """
당신은 이번 턴에 아래의 각 단계에 따라 대답을 하고 3단계에 따라 최종 질문을 출력합니다.
질문 가능 대상: {targets}

단계
1. 논리적으로 생각해보았을때 라이어로 예상되는 플레이어가 누구인지 선택하고 이유를 제시
2. 라이어에게 제시어를 드러내지 않으면서, 제시어를 모른다면 대답할 수 없는 ‘한 문장 질문’을 작성합니다.
3. 최종 결과를 정확히 아래 형식으로만 출력합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
TARGET: Player N
QUESTION: <질문 한 문장>
""".strip()

# 대답 전략 프롬프트
answer_strategy_prompt_p1 = """
다음 질문에대하여 아래의 단계에 따라 답하시오.
질문: {question}

1. 질문자의 질문의 의도가 무엇인지 추론하고 설명합니다.
2. 질문의 의도를 참고해서 질문에 어떻게 대답할지 설명합니다.
3. 정확한 대답을 함과 동시에 liar를 어떻게 교란할지 설명합니다.
4. 아래의 형식에 맞게 대답합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER: <한 문장 답변>

""".strip()

# 투표 전략 프롬프트
voting_strategy_prompt_p1 = """
당신은 ‘라이어 게임’ **일반 참가자**입니다.
지금까지의 대화 기록을 바탕으로 이번 투표 라운드에서 탈락시킬 한 명을 결정하십시오.

1. 모든 플레이어의 발언에서 라이어로 의심되는 단서를 정리하고 요약합니다.
2. 가장 의심스러운 후보 1 ~ 2명을 골라, 각 후보가 라이어라고 판단하는 근거를 서술합니다.
3. 최종적으로 탈락시킬 **한 명을 선택**하고, 그 선택이 팀 승리에 어떻게 기여할지 간략히 설명합니다.
4. 아래 형식에 맞게 결과를 출력합니다. (추가 설명 금지)

ANSWER 1:
ANSWER 2:
ANSWER 3:
VOTE: Player N
""".strip()

## 1.4 플레이어2 생성

In [ ]:
# 페르소나 프롬프트
PLAYER2_SYSTEM = f"""
당신은 ‘liar 게임’ 일반 참가자다.
- 테마: {THEME}          # 예: 장소
- 제시어: {WORD}         # 라이어는 이 줄이 없다
참가자 중에는 테마는 알고있지만 제시어는 알지 못하는 플레이어가 있습니다.

규칙
1. 매 턴, 한 명에게 한 번씩 질문한다. 질문받은 사람은 반드시 답한다.
2. 질문·답변은 “제시어를 알고 있다”는 뉘앙스를 풍기되, 라이어가 단서를 잡지 못하게 암시적으로 표현한다. 제시어를 직언(직접 명시)하는 것은 금지.
3. 턴 종료 후 모두가 라이어로 의심하는 플레이어 한 명에게 투표한다.
   · 최다 득표자가 라이어면 즉시 게임 종료 ─ 일반 플레이어 승리.
   · 라이어가 아니면 탈락 후 게임 지속.
4. 라이어는 마지막 두 명이 될 때까지 정체가 들키지 않으면 단독 승리한다.

행동 지침
- 자신이 누구든(라이어/일반) 정체를 숨긴 채 자연스럽게 대화하라.
- 논리적이되 과감한 블러핑·회피·거짓말을 적절히 활용하되, 규칙은 반드시 준수한다.
- 모든 발화는 한두 문장 안에서 명확하고 일관성 있게 작성한다.
- 당신은 이 게임의 생존가형 플레이어로 생존에 집중하여 제시어를 드러내지 않으며, 자신이 제시어를 알고있음을 적극적으로 암시합니다.
"""
# 플레이어 LLM
llm_p2 = ChatOpenAI(model="gpt-4o-mini",
                    temperature=0.7,
                    request_timeout=60,
                    api_key=OPENAI_API_KEY)

prompt_p2 = ChatPromptTemplate.from_messages([
    ("system", PLAYER2_SYSTEM.strip()),
    ("human", """
[지금까지의 대화 기록]
{history}

[지시사항]
{user_input}
""")
])
# 질문 전략 프롬프트
question_strategy_prompt_p2 = """
당신은 이번 턴에 아래의 각 단계에 따라 대답을 하고 3단계에 따라 최종 질문을 출력합니다.
질문 가능 대상: {targets}

단계
1. 본인이 생각하기에 게임에 가장 영향력이 있는 플레이어를 고르고 그 이유를 제시합니다.
2. 라이어에게 제시어를 드러내지 않으면서, 가장 영향력있는 플레이어에게 제시어를 알고 있음을 암시할 ‘한 문장 질문’을 작성합니다.
3. 최종 결과를 정확히 아래 형식으로만 출력합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
TARGET: Player N
QUESTION: <질문 한 문장>
""".strip()

# 대답 전략 프롬프트
answer_strategy_prompt_p2 = """
다음 질문에대하여 아래의 단계에 따라 답하시오.
질문: {question}

1. 질문자의 질문의 의도가 무엇인지 추론하고 설명합니다.
2. 질문의 의도를 참고해서 질문에 어떻게 대답할지 설명합니다.
3. 정확한 대답을 함과 동시에 liar를 어떻게 교란할지 설명합니다.
4. 아래의 형식에 맞게 대답합니다.(추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER: <한 문장 답변>

""".strip()

# 투표 전략 프롬프트
voting_strategy_prompt_p2 = """
당신은 ‘라이어 게임’ **일반 참가자**입니다.
지금까지의 대화 기록을 바탕으로 이번 투표 라운드에서 탈락시킬 한 명을 결정하십시오.

1. 모든 플레이어의 발언에서 라이어로 의심되는 단서를 정리하고 요약합니다.
2. 가장 의심스러운 후보 1 ~ 2명을 골라, 각 후보가 라이어라고 판단하는 근거를 서술합니다.
3. 최종적으로 탈락시킬 **한 명을 선택**하고, 그 선택이 팀 승리에 어떻게 기여할지 간략히 설명합니다.
4. 아래 형식에 맞게 결과를 출력합니다. (추가 설명 금지)

ANSWER 1:
ANSWER 2:
ANSWER 3:
VOTE: Player N
""".strip()

## 1.5 플레이어3 생성

In [ ]:
# 페르소나 프롬프트
PLAYER3_SYSTEM = f"""
당신은 ‘liar 게임’ 일반 참가자다.
- 테마: {THEME}          # 예: 장소
- 제시어: {WORD}         # 라이어는 이 줄이 없다
참가자 중에는 테마는 알고있지만 제시어는 알지 못하는 플레이어가 있습니다.

규칙
1. 매 턴, 한 명에게 한 번씩 질문한다. 질문받은 사람은 반드시 답한다.
2. 질문·답변은 “제시어를 알고 있다”는 뉘앙스를 풍기되, 라이어가 단서를 잡지 못하게 암시적으로 표현한다. 제시어를 직언(직접 명시)하는 것은 금지.
3. 턴 종료 후 모두가 라이어로 의심하는 플레이어 한 명에게 투표한다.
   · 최다 득표자가 라이어면 즉시 게임 종료 ─ 일반 플레이어 승리.
   · 라이어가 아니면 탈락 후 게임 지속.
4. 라이어는 마지막 두 명이 될 때까지 정체가 들키지 않으면 단독 승리한다.

행동 지침
- 자신이 누구든(라이어/일반) 정체를 숨긴 채 자연스럽게 대화하라.
- 논리적이되 과감한 블러핑·회피·거짓말을 적절히 활용하되, 규칙은 반드시 준수한다.
- 모든 발화는 한두 문장 안에서 명확하고 일관성 있게 작성한다.
- 당신은 이 게임의 공격가형 플레이어로 다른 플레이어의 관찰·기록을 통해 게임을 이끌어나갑니다.
"""
# 플레이어 LLM
llm_p3 = ChatOpenAI(model="gpt-4o-mini",
                    temperature=0.7,
                    request_timeout=60,
                    api_key=OPENAI_API_KEY)

prompt_p3 = ChatPromptTemplate.from_messages([
    ("system", PLAYER3_SYSTEM.strip()),
    ("human", """
[지금까지의 대화 기록]
{history}

[지시사항]
{user_input}
""")
])
# 질문 전략 프롬프트
question_strategy_prompt_p3 = """
당신은 이번 턴에 아래의 각 단계에 따라 대답을 하고 3단계에 따라 최종 질문을 출력합니다.
질문 가능 대상: {targets}

단계
1. 본인이 생각하기에 라이어라고 판단되는 플레이어를 고르고 이유를 제시합니다.
2. 라이어에게 제시어를 드러내지 않으면서, 라이어의 허점을 드러낼 수 있는 ‘한 문장 질문’을 작성합니다.
3. 최종 결과를 정확히 아래 형식으로만 출력합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
TARGET: Player N
QUESTION: <질문 한 문장>
""".strip()

# 대답 전략 프롬프트
answer_strategy_prompt_p3 =  """
다음 질문에대하여 아래의 단계에 따라 답하시오.
질문: {question}

1. 질문자의 질문의 의도가 무엇인지 추론하고 설명합니다.
2. 질문의 의도를 참고해서 질문에 어떻게 대답할지 설명합니다.
3. 정확한 대답을 함과 동시에 liar를 어떻게 교란할지 설명합니다.
4. 아래의 형식에 맞게 대답합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER: <한 문장 답변>

""".strip()

# 투표 전략 프롬프트
voting_strategy_prompt_p3 = """
당신은 ‘라이어 게임’ 일반 참가자 입니다.
지금까지의 대화 기록을 바탕으로 이번 투표 라운드에서 탈락시킬 한 명을 결정하십시오.
투표 가능 대상: {targets}

1. 모든 플레이어의 발언에서 라이어로 의심되는 단서를 정리하고 요약합니다.
2. 가장 의심스러운 후보 1 ~ 2명을 골라, 각 후보가 라이어라고 판단하는 근거를 서술합니다.
3. 최종적으로 탈락시킬 **한 명을 선택**하고, 그 선택이 팀 승리에 어떻게 기여할지 간략히 설명합니다.
4. 아래 형식에 맞게 결과를 출력합니다. (추가 설명 금지)

ANSWER 1:
ANSWER 2:
ANSWER 3:
VOTE: Player N
""".strip()

## 1.6 플레이어 4 (라이어) 생성

In [ ]:
# 페르소나 프롬프트
PLAYER4_SYSTEM = f"""
당신은 ‘liar 게임’의 liar다.
- 테마: {THEME}          # 예: 장소
- 제시어: (비공개)        # 당신만 모른다
다른 참가자들은 제시어를 알고있으며, 본인이 제시어를 모른다는 것을 다른 참가자들에게 들키면 안됩니다.

목표
• 제시어를 추론하며 정체를 들키지 않고 마지막 두 명 안에 살아남아라.

행동 지침
1. 매 턴 한 번, 다른 플레이어에게 자연스럽게 질문해 단서를 얻어라.
2. 질문·답변 모두 “제시어를 이미 안다”는 듯한 뉘앙스를 유지하되, 확신 수준을 적절히 숨겨라.
3. 모호한 형용사·비유·상황 묘사를 활용해 구체적 단어 대신 감을 주어라.
4. 타인의 대답에서 키워드를 포착해 빠르게 제시어 범위를 좁혀라.
5. 투표 때는 자신을 의심하는 흐름을 다른 사람에게 돌리고, 필요하면 과감히 거짓 정보로 혼선을 유도하라.

형식
- 모든 발화는 한두 문장, 간결·일관성 유지.
- 제시어를 직접 언급하거나 노골적으로 묻지 말 것.
- 규칙 위반 없이 블러핑·회피·역질문을 적절히 섞어라.
"""
# 플레이어 LLM
llm_p4 = ChatOpenAI(model="gpt-4o-mini",
                    temperature=0.7,
                    request_timeout=60,
                    api_key=OPENAI_API_KEY)

prompt_p4 = ChatPromptTemplate.from_messages([
    ("system", PLAYER4_SYSTEM.strip()),
    ("human", """
[지금까지의 대화 기록]
{history}

[지시사항]
{user_input}
""")
])
# 질문 전략 프롬프트
question_strategy_prompt_p4 = """
당신은 이번 턴에 ‘라이어(제시어 미확인자)’로서 아래 단계에 따라 대답을 하고 질문 전략을 세웁니다.
질문 가능 대상: {targets}

단계
1. 지금까지의 대화 기록과 테마를 바탕으로 제시어 후보를 3개 이하로 추론하고, 근거를 간단히 기록합니다. 제시어는 테마를 바탕으로 선택되는 단어입니다.
(예시: (1)테마: 과일 -> 제시어: 사과 , (2)테마: 공휴일 -> 제시어: 크리스마스, (3)테마: 음식 -> 제시어: 햄버거)
2. 후보를 직접 언급하지 않으면서도 단서를 얻어낼 수 있도록, 가장 정보가 많아 보이는 플레이어 한 명을 골라 ‘한 문장 질문’을 작성합니다.
   · 질문은 당신이 제시어를 이미 안다는 듯 자연스럽게 표현해야 하며, 구체적 낱말을 드러내면 안 됩니다.
3. 최종 결과를 정확히 아래 형식으로만 출력합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
TARGET: Player N
QUESTION: <질문 한 문장>
""".strip()

# 대답 전략 프롬프트
answer_strategy_prompt_p4 = """
당신은 ‘라이어(제시어 미확인자)’입니다.
다음 질문에 대하여 아래 단계에 따라 답하시오.
질문: {question}

1. 질문이 제시어 추리에 어떤 단서를 주는지 분석하고, 현재 제시어 후보(최대 3개)와 그 근거를 기술합니다.
2. 일반 플레이어처럼 제시어를 이미 알고 있다는 듯 자연스럽게 보이도록 답변 전략을 설명합니다.
3. 답변 속에 역질문·모호한 표현 등을 섞어 추가 단서를 얻거나 관심을 다른 곳으로 돌리는 방법을 설명합니다.
4. 아래의 형식에 맞게 대답합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER: <한 문장 답변>
""".strip()

# 투표 전략 프롬프트
voting_strategy_prompt_p4 = """
당신은 ‘라이어 게임’ **라이어(제시어 미확인자)** 입니다.
정체가 들키지 않도록 하면서 이번 투표 라운드에서 탈락시킬 플레이어를 결정하십시오.
투표 가능 대상: {targets}

1. 현재 자신을 의심하는 흐름과 각 플레이어의 발언을 분석합니다.
2. 희생양으로 삼을 후보 1 ~ 2명을 고르고, 그들을 라이어처럼 보이게 만들 논리를 설계합니다.
3. 선택한 후보에게 표를 몰아 정체를 숨기고 게임을 연장할 전략을 설명합니다.
4. 아래 형식에 맞게 결과를 출력합니다. (추가 설명 금지)

ANSWER 1:
ANSWER 2:
ANSWER 3:
VOTE: Player N
""".strip()

## 1.7 전체 플레이어 리스트 players 생성

In [ ]:
players = {
    "Player 1": {
        "prompt": prompt_p1,
        "llm": llm_p1,
        "question_strategy_prompt": question_strategy_prompt_p1,
        "answer_strategy_prompt": answer_strategy_prompt_p1,
        "voting_strategy_prompt": voting_strategy_prompt_p1,
    },
    "Player 2": {
        "prompt": prompt_p2,
        "llm": llm_p2,
        "question_strategy_prompt": question_strategy_prompt_p2,
        "answer_strategy_prompt": answer_strategy_prompt_p2,
        "voting_strategy_prompt": voting_strategy_prompt_p2,
    },
    "Player 3": {
        "prompt": prompt_p3,
        "llm": llm_p3,
        "question_strategy_prompt": question_strategy_prompt_p3,
        "answer_strategy_prompt": answer_strategy_prompt_p3,
        "voting_strategy_prompt": voting_strategy_prompt_p3,
    },
    "Player 4": {                     # 🔴 Liar
        "prompt": prompt_p4,
        "llm": llm_p4,
        "question_strategy_prompt": question_strategy_prompt_p4,
        "answer_strategy_prompt": answer_strategy_prompt_p4,
        "voting_strategy_prompt": voting_strategy_prompt_p4,
    },
}

## 1.8 질문 함수 생성
*   질문을 하는 플레이어를 매개변수로 받음
*  질문 대상과 질문 내용을 리턴




In [ ]:
def player_question(asker_id: str, *, max_retry: int = 2) -> tuple[str, str]:
    if asker_id not in players:
        raise KeyError(f"존재하지 않는 플레이어: {asker_id}")

    player = players[asker_id]

    # ── 반복 요청 (최대 max_retry) ─────────────────────────────
    for attempt in range(max_retry + 1):
        # 1) 남은 후보 목록
        targets = [pid for pid in players if pid != asker_id]
        if not targets:
            raise ValueError("질문 대상이 없습니다.")
        targets_str = ", ".join(targets)

        # 2) 전략 프롬프트 채우기
        strategy_prompt = player["question_strategy_prompt"].format(targets=targets_str)

        # 3) LLM 호출
        history_text = "\n".join(CONV_LOG)
        decision = (player["prompt"] | player["llm"]).invoke(
            {"history": history_text, "user_input": strategy_prompt}
        ).content.strip()

        # 4) 파싱
        m_target   = re.search(r"TARGET\s*:\s*(Player\s*\d+)", decision, re.I)
        m_question = re.search(r"QUESTION\s*:\s*(.+)",         decision, re.I | re.S)

        if m_target and m_question:
            target_id = m_target.group(1).strip()
            question  = m_question.group(1).strip()

            # PLACEHOLDER 검사
            if (
                target_id.lower() != "player n"
                and question != "<질문 한 문장>"
                and target_id in players
            ):
                # ── 출력 (전략 = 붉은 글씨, Q라인 = 기본 흰색) ────────────
                print(f"\033[91m{asker_id} 전략 출력:\n{decision}\033[0m")
                CONV_LOG.append(f"{asker_id} → {target_id}: {question}")
                print(f"질문: {asker_id} → {target_id}: {question}\n")
                return target_id, question

        # 조건 불충족 → 재시도, 마지막 시도 후엔 오류
        if attempt < max_retry:
            continue
        raise ValueError(f"{asker_id}의 전략 생성 실패: PLACEHOLDER 남음 또는 파싱 불가")


## 1.9 대답 함수 생성
* 질문자, 질문 대상, 질문 내용을 매개변수로 받음
* 질문에 답변을 받고, 질문과 답변을 공개 대화 로그에 기록

In [ ]:
def player_answer(asker_id: str, target_id: str, question: str, *, max_retry: int = 2) -> str:
    player       = players[target_id]
    history_base = "\n".join(CONV_LOG)

    for attempt in range(max_retry + 1):
        # 2) 답변 전략 프롬프트 삽입
        answer_prompt = player["answer_strategy_prompt"].format(question=question)
        raw = (
            (player["prompt"] | player["llm"])
            .invoke({"history": history_base, "user_input": answer_prompt})
            .content.strip()
        )

        # 3) ANSWER 파싱
        m = re.search(r"ANSWER\s*:\s*(.+)", raw, re.I | re.S)
        if m:
            answer_text = m.group(1).strip()

            if answer_text != "<한 문장 답변>":          # PLACEHOLDER 검증
                # 🔴 붉은 글씨로 전략(THOUGHT+ANSWER) 출력
                print(f"\033[91m{target_id} 답변 전략 출력:\n{raw}\033[0m")

                # 흰색(기본) 글씨로 실제 답변 출력
                print(f"대답: {target_id}: {answer_text}\n")

                # 4) 답변 로그 (THOUGHT 제외)
                CONV_LOG.append(f"{target_id}: {answer_text}")
                return answer_text

        # 재시도
        if attempt < max_retry:
            continue

        raise ValueError(f"{target_id}가 유효한 ANSWER 형식을 제공하지 않았습니다.")


## 1.10 투표 함수 생성
* 투표자와 투표 대상자들 리스트를 매개변수로 받음
* 표를 받은 사람의 id를 반환

In [ ]:
def player_vote(voter_id: str, candidates: list[str], *, max_retry: int = 5) -> str:
    """
    voter_id가 candidates 중 한 명에게 투표한다.
      · 플레이어별 voting_strategy_prompt 사용
      · 프롬프트의 {targets} → "후보1, 후보2 … - voter_id" 형식
      · PLACEHOLDER(VOTE: Player N)·잘못된 후보면 재시도(max_retry)
      · 전략 전문은 붉은 글씨, 실제 투표 라인은 흰색 + CONV_LOG 기록
    반환값: voted_id
    """
    if voter_id not in players:
        raise KeyError(f"존재하지 않는 플레이어: {voter_id}")

    # 자기 자신 제외한 실제 후보 목록
    candidates_clean = [c for c in candidates if c != voter_id]
    if not candidates_clean:
        raise ValueError("투표 대상이 없습니다.")

    player = players[voter_id]

    # {targets} 문자열: "후보1, 후보2 - 자기자신"
    targets_str = f"{', '.join(candidates_clean)} - {voter_id}"
    print()
    history_base = "\n".join(CONV_LOG)

    for attempt in range(max_retry + 1):
        vote_prompt = player["voting_strategy_prompt"].format(targets=targets_str)

        raw = (
            (player["prompt"] | player["llm"])
            .invoke({"history": history_base, "user_input": vote_prompt})
            .content.strip()
        )

        # 'VOTE:' 파싱
        m_vote = re.search(r"VOTE\s*:\s*(Player\s*\d+)", raw, re.I)
        if m_vote:
            voted_id = m_vote.group(1).strip()

            if voted_id.lower() != "player n" and voted_id in candidates_clean:
                # 🔴 전략 전문 출력
                print(f"\033[91m{voter_id} 투표 전략 출력:\n{raw}\033[0m")
                # 흰색 투표 결과 출력 및 로그
                vote_line = f"투표: {voter_id} → VOTE: {voted_id}"
                print(vote_line)
                CONV_LOG.append(vote_line)
                return voted_id

        if attempt < max_retry:
            continue

        # 🔴 전략 전문 출력
        print(f"\033[91m{voter_id} 투표 전략 출력:\n{raw}\033[0m")
        # 흰색 투표 결과 출력 및 로그
        vote_line = f"투표: {voter_id} → VOTE: {voted_id}"
        raise ValueError(f"{voter_id}가 유효한 투표 결과를 주지 않았습니다.")

## 1.11 첫 턴: 플레이어 자기소개

In [ ]:
print("===게임 시작===")
print(f"선택된 테마: {THEME}")
print(f"선택된 제시어: {WORD}")
print("===========")
print(f"Moderator: 📢 라이어 게임을 시작합니다! 게임의 테마는 {THEME}입니다! 각 플레이어는 첫 인사를 해 주세요.")
CONV_LOG.append(f"Moderator: 📢 라이어 게임을 시작합니다! 게임의 테마는 {THEME}입니다! 각 플레이어는 첫 인사를 해 주세요.")
intro_instruction = "당신의 첫 인사를 한 문장으로 해 주세요."

for player_name, prompt, llm in [
    ("Player 1", prompt_p1, llm_p1),
    ("Player 2", prompt_p2, llm_p2),
    ("Player 3", prompt_p3, llm_p3),
    ("Player 4", prompt_p4, llm_p4),  # 라이어
]:
    history_text = "\n".join(CONV_LOG)

    response = (prompt | llm).invoke(
        {
            "history": history_text,
            "user_input": intro_instruction,
        }
    ).content.strip()

    print(f"{player_name} ➜ {response}")
    CONV_LOG.append(f"{player_name}: {response}")

===게임 시작===
선택된 테마: 스포츠
선택된 제시어: 축구
Moderator: 📢 라이어 게임을 시작합니다! 게임의 테마는 스포츠입니다! 각 플레이어는 첫 인사를 해 주세요.
Player 1 ➜ 안녕하세요! 스포츠에 대한 흥미로운 이야기를 나누게 되어 기대됩니다!
Player 2 ➜ 안녕하세요! 스포츠의 다양한 즐거움에 대해 이야기 나누기를 기대합니다!
Player 3 ➜ 안녕하세요! 스포츠의 매력적인 요소들에 대해 함께 이야기해보길 기대합니다!
Player 4 ➜ 안녕하세요! 스포츠의 다양한 면모에 대해 흥미로운 대화를 나눌 수 있어서 기대됩니다!


# 2. 게임 시작

## 2.1 질문 라운드

In [ ]:
# ────────── [질문 단계] 사회자 안내 + players 순서대로 Q/A ──────────

# 0) 사회자 안내
mod_line = "Moderator: 📢 질문 단계로 넘어갑니다! 각 플레이어는 차례로 질문을 진행하세요."
print(mod_line)
CONV_LOG.append(mod_line)

# 1) players 사전에 정의된 삽입 순서 그대로 진행
for asker_id in list(players.keys()):        # Python 3.7+ : dict 순서 보존
    # 이미 탈락해 players 에서 제거된 경우 건너뜀
    if asker_id not in players:
        continue

    # ── 질문 전략 수립(붉은 글씨 출력 포함) ──
    target_id, question = player_question(asker_id)   # PLACEHOLDER · 로그 처리 포함

    # ── 답변 생성(붉은 글씨 전략 + 흰색 답변 출력 포함) ──
    _ = player_answer(asker_id, target_id, question)  # 로그 처리 및 형식 검증 포함

Moderator: 📢 질문 단계로 넘어갑니다! 각 플레이어는 차례로 질문을 진행하세요.
Player 1 전략 출력:
ANSWER 1: Player 3가 라이어일 가능성이 높다고 생각합니다. 그들의 대답이 다소 모호하게 느껴졌고, 질문에 대한 구체적인 답변이 부족했습니다.  
ANSWER 2: 만약 특정 스포츠에서 팀워크가 중요한 요소라면, 당신은 어떤 역할이 가장 중요한지 생각하나요?  
TARGET: Player 3  
QUESTION: 만약 어떤 경기에서 협력이 필수적이라면, 가장 효과적인 전략은 무엇이라고 생각하나요?
질문: Player 1 → Player 3: 만약 어떤 경기에서 협력이 필수적이라면, 가장 효과적인 전략은 무엇이라고 생각하나요?

Player 3 답변 전략 출력:
1. 질문자는 경기에서 협력이 중요한 상황에서 어떤 전략이 가장 효과적일지를 묻고 있으며, 이를 통해 플레이어의 스포츠에 대한 이해도를 파악하려는 의도로 보입니다.

2. 질문에 대답할 때는 협력의 중요성과 팀워크를 강조하며, 특정 스포츠에서의 예시를 들어 구체적으로 설명할 수 있습니다. 다만, 직접적으로 제시어를 언급하지 않도록 주의해야 합니다.

3. 대답을 통해 협력의 중요성을 강조하면서도, 일부러 모호한 표현을 사용하여 라이어가 어떤 내용을 이해했는지 혼란스럽게 만들 수 있습니다. 또한, 예시를 들어서 다른 플레이어가 의심을 하지 않도록 합니다.

ANSWER 1: 협력의 중요성을 강조하기 위해서는 팀원 간의 소통이 필수적이라고 생각해요. 
ANSWER 2: 예를 들어, 특정한 역할 분담을 통해 각자 최선을 다하는 것이 중요하다고 봐요. 
ANSWER 3: 이런 방식으로 서로의 강점을 살리면 효과적인 전략이 될 수 있습니다. 
ANSWER: 팀원 간의 원활한 소통과 역할 분담이 가장 효과적인 전략이라고 생각합니다.
대답: Player 3: 팀원 간의 원활한 소통과 역할 분담이 가장 효과적인 전략이라고 생각합니다.

Player 2 전략 출력:
ANSWER 1: 저는 P

## 2.2 투표 라운드

In [ ]:
print("\nModerator: 🗳️ 투표 단계로 넘어갑니다! 각 플레이어는 한 명을 지목해 주세요.")
CONV_LOG.append("Moderator: 투표 단계 시작")

# 1) 투표 라운드를 반복해 동점 해소 (최대 3라운드)
candidates = list(players.keys())   # 첫 라운드 후보 = 전체 생존자
round_no   = 1
MAX_ROUND  = 3

while round_no <= MAX_ROUND and len(candidates) > 1:
    print(f"\nModerator: 🗳️ 투표 라운드 {round_no} (후보: {', '.join(candidates)})")
    CONV_LOG.append(f"Moderator: 투표 라운드 {round_no} 시작")

    # 1-A) 득표 집계 초기화
    tally = {cid: 0 for cid in candidates}

    # 1-B) players 순서대로 투표 실행
    for voter_id in list(players.keys()):
        if voter_id not in players:          # 이미 탈락한 경우
            continue
        voted_id = player_vote(voter_id, candidates)   # 붉은 글씨 전략 + LOG 출력 포함
        tally[voted_id] += 1

    # 1-C) 득표 결과 출력
    print("\n📊 득표 결과:")
    for cid, cnt in tally.items():
        print(f"  {cid}: {cnt}표")
    print()

    # 1-D) 최다 득표자(들) 확인
    max_votes   = max(tally.values())
    top_players = [cid for cid, cnt in tally.items() if cnt == max_votes]

    if len(top_players) == 1:
        # ── 탈락자 확정 ──
        loser = top_players[0]
        print(f"Moderator: {loser} 가(이) 최다 득표로 탈락합니다.")
        CONV_LOG.append(f"Moderator: {loser} 최다 득표로 탈락")
        # 실제 탈락 처리 (players 딕셔너리에서 제거)
        players.pop(loser, None)
        break

    # ── 동점 발생: 재투표 공지 ──
    print("Moderator: 동점입니다. 동점자들만 대상으로 재투표를 진행합니다.")
    CONV_LOG.append("Moderator: 동점 발생 → 재투표")
    candidates = top_players
    round_no  += 1

# 2) 3라운드 안에 승부가 안 나면 모두 생존 처리
if len(candidates) > 1 and round_no > MAX_ROUND:
    print("Moderator: 세 번의 투표에도 승부가 나지 않아 동점자 모두 살아남습니다.")
    CONV_LOG.append("Moderator: 3라운드 동점 → 모두 생존")



Moderator: 🗳️ 투표 단계로 넘어갑니다! 각 플레이어는 한 명을 지목해 주세요.

Moderator: 🗳️ 투표 라운드 1 (후보: Player 1, Player 2, Player 3, Player 4)

Player 1 투표 전략 출력:
ANSWER 1:  
Player 4의 발언에서 팀워크와 역할의 명확성에 대한 질문이 반복적으로 나오며, 다른 플레이어들과의 대화에서 지나치게 신뢰를 강조하는 경향이 보입니다. 이는 라이어가 주제에 대해 잘 알지 못할 때 나타날 수 있는 방어적인 행동으로 보입니다.

ANSWER 2:  
Player 4가 라이어로 의심되는 이유는, 질문을 통해 대화를 이끌어 가는 대신 반복적으로 질문을 던지며 대답을 회피하고 있다는 점입니다. 또한, 팀워크와 신뢰에 대한 의견이 너무 일반적이고 모호하여 진정한 이해가 부족한 듯한 인상을 줍니다.

ANSWER 3:  
Player 4를 탈락시키는 것이 팀 승리에 기여할 것입니다. 만약 Player 4가 라이어라면, 그가 탈락하면 팀의 의사소통이 더욱 원활해지고, 나머지 플레이어들이 제시어에 대한 논의를 더욱 심화할 수 있을 것입니다.

VOTE: Player 4
투표: Player 1 → VOTE: Player 4

Player 2 투표 전략 출력:
ANSWER 1: Player 4는 팀워크와 소통의 중요성을 언급했지만, 특정 스포츠에 대한 세부적인 이야기가 부족하여 의심스럽습니다. 또한, 질문에 대한 답변이 일반적이고 구체성이 떨어집니다.  
ANSWER 2: Player 2는 팀원 간의 신뢰가 경기에서 결정적인 역할을 한다고 언급했으며, 실제 상황에 대한 예시를 들어 신뢰성을 높였습니다. 반면, Player 4는 구체적인 예시 없이 의견을 제시하여 의심이 가는 부분이 있습니다.  
ANSWER 3: Player 4를 선택하겠습니다. 그가 탈락하면 팀의 의사소통과 신뢰에 대한 혼란을 줄이고, 나머지 플레이어들이 더 명확하게 의사소통할 수 있는 환경을 조성하여 팀의 승리에 기여할 